In [1]:
import pandas as pd
import datetime
from IPython.display import display, Markdown

# --- 1. Load and Prepare the Database ---
df_stations = pd.read_csv("aerodrome_database.csv")

# Focus on essential columns and clean up
df_stations = df_stations[['icao', 'iata', 'name', 'latitude', 'longitude', 'elevation_ft', 'country']]
df_stations = df_stations[df_stations['country'] == 'CA'].copy()
df_stations['icao'] = df_stations['icao'].str.upper().str.strip()
df_stations['iata'] = df_stations['iata'].str.upper().str.strip()

# --- 2. Conceptual Data Retrieval Function ---
# (Simulates fetching dynamic weather data for a fixed station)
def query_wmo_data(date_time: datetime.datetime, station_data: pd.Series) -> dict:
    """
    Simulates querying a large meteorological dataset using a station's static data.
    """
    lat = station_data['latitude']
    lon = station_data['longitude']
    alt = station_data['elevation_ft']
    name = station_data['name']
    icao = station_data['icao']

    print(f"\n--- Processing Query Against WMO-Style Dataset ---")
    print(f"Searching for data for: {name} ({icao})")

    # Simulate weather conditions based on a simple heuristic (e.g., warmer in the south)
    base_temp = 10 + (60 - lat) * 0.5  # Simple temp decrease with higher latitude

    simulated_data = {
        'WMO_Block_Estimate': 71,  # All Canadian synoptic stations fall in this block
        'Time_of_Observation': date_time.strftime('%Y-%m-%d %H:%M UTC'),
        'Station_ICAO': icao,
        'Station_Name': name,
        'Latitude': f"{lat}°N",
        'Longitude': f"{lon}°E",
        'Altitude_m': f"{round(alt * 0.3048, 1)} m", # Convert feet to meters
        'Interpolated_Temperature_C': round(base_temp, 1),
        'Interpolated_Pressure_hPa': 1012.3,
        'Wind_Speed_ms': 5.2,
        'Data_Source': 'Simulated Global Weather Model'
    }

    return simulated_data

In [2]:
# --- 3. Get User Input (Station and Time) ---

# Get Station Identifier
while True:
    station_id = input("Enter Airport ICAO or IATA code (e.g., CYYZ or YYZ): ").strip().upper()

    # Try looking up by ICAO first, then IATA
    station_match = df_stations[df_stations['icao'] == station_id]
    if station_match.empty:
        station_match = df_stations[df_stations['iata'] == station_id]

    if not station_match.empty:
        # Get the first match's data
        user_station_data = station_match.iloc[0]
        break
    else:
        print(f"❌ Station code '{station_id}' not found in the Canadian database. Try 'CYYZ' or 'YUL'.")


# Get Date and Time
print(f"\n--- Enter Observation Time for {user_station_data['name']} ---")
while True:
    try:
        date_str = input("Enter Date (YYYY-MM-DD): ")
        time_str = input("Enter Time (HH:MM in 24h format): ")
        date_time_str = f"{date_str} {time_str}"
        user_datetime = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        break
    except ValueError:
        print("❌ Invalid date or time format. Please use YYYY-MM-DD and HH:MM. Try again.")


Enter Airport ICAO or IATA code (e.g., CYYZ or YYZ): CYYZ

--- Enter Observation Time for Toronto Lester B. Pearson International Airport ---
Enter Date (YYYY-MM-DD): 2025-10-01
Enter Time (HH:MM in 24h format): 03:35


In [3]:
# --- 4. Execute the Conceptual Query and Display Results ---
try:
    # Pass the gathered time and static station data to the conceptual function
    result_data = query_wmo_data(user_datetime, user_station_data)

    # --- Display Results ---
    markdown_output = (
        "## ✅ WMO-Style Data Retrieval Summary\n"
        f"Query Time: **{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}**\n\n"
        "| Parameter | Value |\n"
        "| :--- | :--- |\n"
    )

    for key, value in result_data.items():
        markdown_output += f"| **{key.replace('_', ' ').title()}** | {value} |\n"

    display(Markdown(markdown_output))

except Exception as e:
    # Fallback error message
    display(Markdown(f"## ❌ An Error Occurred\nAn unexpected error occurred during data processing: `{e}`"))


--- Processing Query Against WMO-Style Dataset ---
Searching for data for: Toronto Lester B. Pearson International Airport (CYYZ)


## ✅ WMO-Style Data Retrieval Summary
Query Time: **2025-10-27 15:37:53**

| Parameter | Value |
| :--- | :--- |
| **Wmo Block Estimate** | 71 |
| **Time Of Observation** | 2025-10-01 03:35 UTC |
| **Station Icao** | CYYZ |
| **Station Name** | Toronto Lester B. Pearson International Airport |
| **Latitude** | 43.6772°N |
| **Longitude** | -79.6306°E |
| **Altitude M** | 173.4 m |
| **Interpolated Temperature C** | 18.2 |
| **Interpolated Pressure Hpa** | 1012.3 |
| **Wind Speed Ms** | 5.2 |
| **Data Source** | Simulated Global Weather Model |


In [4]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import display, Markdown

# --- 1. Haversine Distance Function ---
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on the Earth specified in decimal degrees (returns km).
    """
    R = 6371  # Earth radius in kilometers

    # Convert degrees to radians
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return c

# --- 2. Conceptual Data Retrieval Function ---
def query_wmo_data_arbitrary(date_time: datetime.datetime, latitude: float, longitude: float, altitude: float) -> dict:
    """
    Simulates querying a large, gridded meteorological dataset for arbitrary coordinates.
    """
    print(f"\n--- Generating Simulated Weather Data for Arbitrary Point ---")

    # Simulate weather conditions based on a simple heuristic
    # Temperature adjustment based on altitude (lapse rate approx 6.5C per 1000m)
    alt_km = altitude / 1000.0
    base_temp = 15.0  # Base temperature at sea level
    temp = base_temp - (alt_km * 6.5)

    simulated_data = {
        'WMO_Block_Estimate': 71,  # Assuming location is generally North American
        'Time_of_Observation': date_time.strftime('%Y-%m-%d %H:%M UTC'),
        'Interpolated_Temperature_C': round(temp, 1),
        'Interpolated_Pressure_hPa': 1012.0 - (alt_km * 10), # Simple pressure drop
        'Wind_Speed_ms': 6.0,
        'Total_Precipitation_mm_past_6h': 0.0,
        'Data_Source': 'Simulated Global Reanalysis Model'
    }

    return simulated_data

# --- 3. Load and Prepare the Database ---
df_stations = pd.read_csv("aerodrome_database.csv")
df_stations = df_stations[df_stations['country'] == 'CA'].copy()
df_stations.rename(columns={'latitude': 'st_lat', 'longitude': 'st_lon', 'elevation_ft': 'st_alt'}, inplace=True)
df_stations['st_alt_m'] = df_stations['st_alt'] * 0.3048

In [5]:
# --- 4. Get Arbitrary User Input (t, lat, lon, alt) ---
print("--- Enter Arbitrary Observation Point ---")

# Get Date and Time
while True:
    try:
        date_str = input("Enter Date (YYYY-MM-DD): ")
        time_str = input("Enter Time (HH:MM in 24h format): ")
        date_time_str = f"{date_str} {time_str}"
        user_datetime = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        break
    except ValueError:
        print("❌ Invalid date or time format. Please use YYYY-MM-DD and HH:MM. Try again.")

# Get Location
while True:
    try:
        user_lat = float(input("Enter Latitude (e.g., 50.0): "))
        user_lon = float(input("Enter Longitude (e.g., -100.0): "))
        user_alt = float(input("Enter Altitude in meters (e.g., 500): "))

        if -90 <= user_lat <= 90 and -180 <= user_lon <= 180:
            break
        else:
            print("❌ Invalid latitude or longitude range. Try again.")
    except ValueError:
        print("❌ Invalid input. Please enter a numerical value for location. Try again.")

--- Enter Arbitrary Observation Point ---
Enter Date (YYYY-MM-DD): 2025-10-01
Enter Time (HH:MM in 24h format): 03:35
Enter Latitude (e.g., 50.0): 35.45
Enter Longitude (e.g., -100.0): -97.85
Enter Altitude in meters (e.g., 500): 9000


In [6]:
# --- 5. Find Nearest Canadian Station for Context ---
print("\n--- Finding Nearest Canadian Aerodrome in CSV Database ---")

# Apply the Haversine calculation
df_stations['distance_km'] = haversine(
    user_lat, user_lon,
    df_stations['st_lat'], df_stations['st_lon']
)

# Find the station with the minimum distance
nearest_station = df_stations.loc[df_stations['distance_km'].idxmin()]
dist_km = round(nearest_station['distance_km'], 2)

# --- 6. Execute Conceptual Query ---
try:
    # Run the conceptual weather retrieval using the arbitrary input
    weather_data = query_wmo_data_arbitrary(user_datetime, user_lat, user_lon, user_alt)

    # --- 7. Display Results ---
    markdown_output = (
        "## ✅ WMO-Style Data Retrieval Summary\n"
        f"Query Time: **{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}**\n\n"
        "### Arbitrary Observation Point\n"
        "| Parameter | Value |\n"
        "| :--- | :--- |\n"
        f"| **Latitude** | {user_lat}°N |\n"
        f"| **Longitude** | {user_lon}°E |\n"
        f"| **Altitude** | {user_alt} m |\n"
        f"| **Observation Time (t)** | {user_datetime.strftime('%Y-%m-%d %H:%M UTC')} |\n"
        "|---|---|\n"

        "### Nearest Reporting Station (from CSV for Context)\n"
        "| Parameter | Value |\n"
        "| :--- | :--- |\n"
        f"| **Station Name** | {nearest_station['name']} ({nearest_station['icao']}) |\n"
        f"| **Station Coordinates** | {nearest_station['st_lat']}°N, {nearest_station['st_lon']}°E |\n"
        f"| **Distance** | **{dist_km} km** |\n"
        "|---|---|\n"

        "### Conceptual Weather Data Retrieval\n"
        "| Parameter | Value |\n"
        "| :--- | :--- |\n"
    )

    for key, value in weather_data.items():
        markdown_output += f"| **{key.replace('_', ' ').title()}** | {value} |\n"

    display(Markdown(markdown_output))

except Exception as e:
    # Fallback error message
    display(Markdown(f"## ❌ An Error Occurred\nAn unexpected error occurred during data processing: `{e}`"))


--- Finding Nearest Canadian Aerodrome in CSV Database ---

--- Generating Simulated Weather Data for Arbitrary Point ---


## ✅ WMO-Style Data Retrieval Summary
Query Time: **2025-10-27 15:44:57**

### Arbitrary Observation Point
| Parameter | Value |
| :--- | :--- |
| **Latitude** | 35.45°N |
| **Longitude** | -97.85°E |
| **Altitude** | 9000.0 m |
| **Observation Time (t)** | 2025-10-01 03:35 UTC |
|---|---|
### Nearest Reporting Station (from CSV for Context)
| Parameter | Value |
| :--- | :--- |
| **Station Name** | Windsor Airport (CYQG) |
| **Station Coordinates** | 42.27560043334961°N, -82.95559692382812°E |
| **Distance** | **1493.17 km** |
|---|---|
### Conceptual Weather Data Retrieval
| Parameter | Value |
| :--- | :--- |
| **Wmo Block Estimate** | 71 |
| **Time Of Observation** | 2025-10-01 03:35 UTC |
| **Interpolated Temperature C** | -43.5 |
| **Interpolated Pressure Hpa** | 922.0 |
| **Wind Speed Ms** | 6.0 |
| **Total Precipitation Mm Past 6H** | 0.0 |
| **Data Source** | Simulated Global Reanalysis Model |
